In [11]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# このSprintについて

## Sprintの目的
* スクラッチを通してリカレントニューラルネットワークの基礎を理解する

## どのように学ぶか
* スクラッチでリカレントニューラルネットワークの実装を行います。

# リカレントニューラルネットワークスクラッチ

リカレントニューラルネットワーク（RNN） のクラスをスクラッチで作成していきます。
NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。

フォワードプロパゲーションの実装を必須課題とし、バックプロパゲーションの実装はアドバンス課題とします。

クラスの名前はScratchSimpleRNNClassifierとしてください。クラスの構造などは以前のSprintで作成したScratchDeepNeuralNetrowkClassifierを参考にしてください。



##【問題1】SimpleRNNのフォワードプロパゲーション実装
SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。


フォワードプロパゲーションの数式は以下のようになります。ndarrayのshapeがどうなるかを併記しています。


バッチサイズをbatch_size、入力の特徴量数をn_features、RNNのノード数をn_nodesとして表記します。活性化関数はtanhとして進めますが、これまでのニューラルネットワーク同様にReLUなどに置き換えられます。


初期状態 $h0$ は全て0とすることが多いですが、任意の値を与えることも可能です。


上記の処理を系列数n_sequences回繰り返すことになります。RNN全体への入力 
x
 は(batch_size, n_sequences, n_features)のような配列で渡されることになり、そこから各時刻の配列を取り出していきます。


分類問題であれば、それぞれの時刻のhに対して全結合層とソフトマックス関数（またはシグモイド関数）を使用します。タスクによっては最後の時刻のhだけを使用することもあります。

In [69]:
class SimpleRNN:
    def __init__(self, n_nodes, initializer, optimizer, activation, debug=False):
        # ノードの最適化・初期化インスタンスを作成
        self.optimizer = optimizer
        self.initializer = initializer
        self.activation  = activation

        self.n_nodes = n_nodes
        self.debug = debug

        # ノードのバイアスを初期化
        if self.debug:
          self.b = np.array([1, 1, 1, 1])
        else:
          self.b = self.initializer.B(n_nodes, )

    def forward(self, X):
        """
        X: 入力(batch_size, n_sequences, n_features)

        """
        batch_size  = X.shape[0]
        n_sequences = X.shape[1]
        n_features  = X.shape[2]

        if self.debug:
          self.w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100
          self.w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100
        else:
          self.w_x = self.initializer.W(n_features, self.n_nodes)
          self.w_h = self.initializer.W(self.n_nodes, self.n_nodes)

        h = np.zeros((n_sequences+1, batch_size, self.n_nodes))

        for i in range(n_sequences):
           a = X[:, i, :] @ self.w_x + h[i, :, :] @ self.w_h + self.b
           h[i+1, :, :] = self.activation.forward(a)
        print(h)
        return h[-1, :, :]

    def backward(self, dA):
        """
        実装なし
        """
        
        return dZ

初期化クラス

In [38]:
class SimpleInitializer:
    def __init__(self, sigma):
        self.sigma = sigma
        
    def W(self, n_nodes1, n_nodes2):
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        return W
    
    def B(self, n_nodes2):
        B = self.sigma * np.random.randn(n_nodes2, )
        return B

最適化手法

In [3]:
class SGD:
    def __init__(self, lr):
        self.lr = lr
    def update(self, layer):
        layer.B -= self.lr * layer.dB
        layer.W -= self.lr * layer.dW

活性化関数

In [4]:
class Sigmoid:
    """
    Sigmoid関数のクラス
    """  
    def __init__(self):
        pass
    
    def forward(self, X):
        self.A = X
        return 1 / (1 + np.exp(-X))
    
    def backward(self, X):
        return X * (1- self.forward(self.A)) * self.forward(self.A)

In [6]:
class Tanh:
    """
    Tanh関数のクラス
    """
    def __init__(self):
        pass
    
    def forward(self, X):
        self.A = X
        return np.tanh(X)
    
    def backward(self, X):
        return X * (1 - self.forward(self.A)**2)

In [7]:
class Softmax:
    def __init__(self):
        pass
    
    def forward(self, X):
        X = X - np.max(X)
        return np.exp(X) / np.sum(np.exp(X), axis=1, keepdims=True)
    
    def backward(self, X, y):
        batch_size = len(X)
        delta = 1e-7
        
        self.loss = -np.sum(y * np.log(X+delta)) / batch_size
        return X - y

In [9]:
class ReLU:
    """
    ReLU関数のクラス
    """
    def forward(self, X):
        self.A = X
        return np.maximum(0, X)
    
    def backward(self, X):
        return X * (self.A > 0)

## 【問題2】小さな配列でのフォワードプロパゲーションの実験
小さな配列でフォワードプロパゲーションを考えてみます。

入力x、初期状態h、重みw_xとw_h、バイアスbを次のようにします。

ここで配列xの軸はバッチサイズ、系列数、特徴量数の順番です。

```python
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)
```

フォワードプロパゲーションの出力が次のようになることを作成したコードで確認してください。

```python
h = np.array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]]) # (batch_size, n_nodes)
```

In [12]:
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)

フォワード部分のみ計算

In [60]:
#１系列目
activation = Tanh()
a1 = x[:, 0, :] @ w_x + h @ w_h + b
h1 = activation.forward(a1)
#２系列目
a2 = x[:, 1, :] @ w_x + h1 @ w_h + b
h2 = activation.forward(a2)
#３系列目
a3 = x[:, 2, :] @ w_x + h2 @ w_h + b
h3 = activation.forward(a3)
h3

print(h1)
print(h2)
print(h3)

[[0.76188798 0.76213958 0.76239095 0.76255841]]
[[0.792209   0.8141834  0.83404912 0.84977719]]
[[0.79494228 0.81839002 0.83939649 0.85584174]]


In [72]:
#初期化クラス
initializer = SimpleInitializer(sigma=0.1)
#最適化クラス
optimizer   = SGD(lr=0.1)
#活性化関数クラス
activation  = Tanh()

#モデル作成
rnn = SimpleRNN(n_nodes, initializer=initializer, optimizer=optimizer, activation=activation, debug=True)
rnn.forward(x)

[[[0.         0.         0.         0.        ]]

 [[0.76188798 0.76213958 0.76239095 0.76255841]]

 [[0.792209   0.8141834  0.83404912 0.84977719]]

 [[0.79494228 0.81839002 0.83939649 0.85584174]]]


array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]])

## 【問題3】（アドバンス課題）バックプロパゲーションの実装
バックプロパゲーションを実装してください。

RNNの内部は全結合層を組み合わせた形になっているので、更新式は全結合層などと同様です。

実装なし・・・